# PINN pour un circuit RC-AC avec une perte physique 
### Decomment 2nd block of code for consistent initialising seed
### Network trained with dataset used ***without DataLoader***

In [ ]:
import torch
print(torch.__version__)


<u>Import nécessaire :</u>

In [ ]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import csv 
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
import random


In [ ]:
# Initialise random seed for model weights and activations

def set_seed(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

    # If using torch.backends (optional for CPU, more relevant for CUDA)
    if torch.backends.mps.is_available():
        torch.use_deterministic_algorithms(True)

set_seed(42)

In [ ]:
# A  COMPLETER AVEC LE NOM DU FICHIER CSV
df = pd.read_csv("rc_ac_results_800.csv")
#print(df)
print(df.shape)

In [ ]:

# Data normalisation
X = df[ [ "R" , "C" , "Vin" , "Frequency" ] ].values
Y = df[ [ "Magnitude", "Phase" ] ].values

scaler_x = StandardScaler()
scaler_y = StandardScaler()

X_tensor = torch.tensor( scaler_x.fit_transform(X) , dtype = torch.float32 )
Y_tensor = torch.tensor( scaler_y.fit_transform(Y) , dtype = torch.float32 )

#print(X_tensor[:,0])
#print(Y_tensor)

dataset = TensorDataset( X_tensor , Y_tensor )

#loader = DataLoader(dataset, batch_size=32, shuffle=True)

R = X_tensor[:,0]
C = X_tensor[:,1]
Frequency = X_tensor[:,3]
print( R )
print( C )

print( ( R*C ).shape )

In [ ]:
class RegressionModel(nn.Module):
    def __init__( self , in_features , out_features = 2 ):
        super().__init__()

        """"
        nn.Linear( a , b ) crée une couche fully connected

            a : neurones en input
            b : neurones en output
        """
        self.fc1 = nn.Linear( in_features , 8 )
        self.fc2 = nn.Linear( 8 , 16 )
        self.fc3 = nn.Linear( 16 , 64 )
        self.fc4 = nn.Linear( 64 , 32 )
        self.fc5 = nn.Linear( 32 , 8 )
        self.fc6 = nn.Linear( 8 , 2 )
        


    def forward( self , x ):
        x = F.relu( self.fc1( x ) )
        x = F.relu( self.fc2( x ) )
        x = F.relu( self.fc3( x ) )
        x = F.relu( self.fc4( x ) )
        x = F.relu( self.fc5( x ) )
        x = self.fc6( x )  # No activation on output for regression
        return x

In [ ]:
in_features = 4
out_features = 2


model = RegressionModel( in_features , out_features )

criterion = nn.MSELoss()
optimizer = optim.Adam( model.parameters() , lr = 0.001 )


In [ ]:
# Function to compute the PDE residual: 

def pde_residual( model , x ):
    # Make sure x is set to require gradients for derivative calculations.
    x = x.clone().detach().requires_grad_( True )
    u = model( x )
    
    #t = x[ :, 0]
    # Compute the first derivative, du/dt.
    #u_x = ( torch.autograd.grad( u , t , grad_outputs = torch.ones_like(u) , create_graph = True )[ 0 ] )[ :,0 ]
    
    """print( u_x.shape )
    print( x.shape )
    print( u[:,0].shape )"""

    R = x[:, 0]
    C = x[:, 1]
    Vin = x[:, 2]
    Frequency = x[:, 3]
    w = 2 * torch.pi * Frequency

    mag = u[ :, 0 ]
    phase = u[ :, 1 ]

    """print(R.shape)
    print(C.shape)
    print(U.shape)"""

    # Formules physiques
    mag_2 = 1 / torch.sqrt( 1 + ( w * R * C ) ** 2 )
    phase_2 = -torch.atan( w * R * C )

    
    residual_mag = (mag - mag_2 ) ** 2
    residual_phase = (phase - phase_2 ) ** 2
    residual = ( residual_mag + residual_phase )
    return torch.mean( residual )


#pde_test = pde_residual(model, X_tensor)
#print(pde_test)
#print(pde_test.shape)



In [ ]:
nb_epochs = 300
losses = []
# Before training
#set_seed(42)
model.train()
for i in range(nb_epochs):
    """
        for X_batch, Y_batch in loader:
        
        outputs = model(X_batch)
        loss = criterion(outputs, Y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    """
    

    # Physics loss
    loss_physics = pde_residual( model , X_tensor )

    # MSE loss
    outputs = model( X_tensor )
    loss_mse = criterion( outputs , Y_tensor ) 

    # final loss
    loss = loss_mse + loss_physics

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    losses.append( loss.item() )


    print( f'Epoch [{i+ 1}], Loss: {loss.item():.9f}' )

# Plot the loss curve
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.grid(True)
plt.show()

   

In [ ]:
model.eval()

# No gradient calculation during evaluation
with torch.no_grad():
    # Forward pass
    predictions = model( X_tensor )  # Assuming X_tensor is your input data

    # Calculate the loss (optional)
    loss = criterion( predictions , Y_tensor )
    print( f"Evaluation Loss: {loss.item():.3f}" )

    predictions_original = scaler_y.inverse_transform( predictions.numpy() )

    # Print the inverse transformed predictions
    print( "Inverse Predictions: \n", predictions_original )

In [ ]:
# Dénormalisation des prédictions et des vraies valeurs
predictions_original = scaler_y.inverse_transform(predictions.numpy())
Y_true_original = scaler_y.inverse_transform(Y_tensor.numpy())

# Dénormalisation de la fréquence
frequencies = scaler_x.inverse_transform(X_tensor.numpy())[:, 3]

# Tracé magnitude réelle vs prédite
plt.figure( figsize = ( 10 , 5 ) )
plt.plot( frequencies, Y_true_original[:, 0] , label="True Magnitude" , linewidth=2 )
plt.plot( frequencies, predictions_original[:, 0] , 'o' , label="Predicted Magnitude" , markersize=4 , color='red' )

plt.xlabel("Frequency (Hz)")
plt.ylabel("Magnitude")
plt.title("Magnitude vs Frequency (données réelles)")

plt.legend()

plt.grid(True)

plt.show()


In [ ]:
# Print model's state_dict
print( "Model's state_dict:" )
for param_tensor in model.state_dict():
    print( param_tensor, "\t", model.state_dict()[param_tensor].size() )

# Print optimizer's state_dict
"""print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])"""



In [ ]:
# Defining model name and save the model

name = "My_model"    # À changer selon l'utilisateur
torch.save( model.state_dict() , name )

# Load the saved model and evaluate

my_model = RegressionModel( in_features , out_features )
my_model.load_state_dict( torch.load( name , weights_only = True ) )
my_model.eval()


with torch.no_grad():
    # Forward pass
    predictions = my_model( X_tensor )  

    # Calculate the loss (optional)
    loss = criterion( predictions , Y_tensor )
    #print( f"Evaluation Loss: {loss.item():.3f}" )

    print(f"Evaluation Loss after loading: {loss.item():.5f}")
    #print(f"Epoch {i+1}: total={loss.item():.5f}, mse={loss_mse.item():.5f}, phys={loss_physics.item():.5f}")
